# Luxury Loan Portfolio Path

In [102]:
import pandas as pd
import matplotlib
import numpy as np
import sklearn
from sklearn import svm
from sklearn import metrics
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

In [3]:
df_acct = pd.read_csv('data-set/completedacct.csv')
df_card = pd.read_csv('data-set/completedcard.csv')
df_client = pd.read_csv('data-set/completedclient.csv')
df_disposition = pd.read_csv('data-set/completeddisposition.csv')
df_district = pd.read_csv('data-set/completeddistrict.csv')
df_loan = pd.read_csv('data-set/completedloan.csv')
df_order = pd.read_csv('data-set/completedorder.csv')
df_trans = pd.read_csv('data-set/completedtrans.csv', low_memory=False)
# df_crm_call = pd.read_csv('data-set/CRM Call Center Logs.csv')
# df_crm_events = pd.read_csv('data-set/CRM Events.csv')
# df_crm_reviews = pd.read_csv('data-set/CRM Reviews.csv')
df_luxury_loans = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')

#### Level 1: Exploring (1 point per task)

1. What are the min, max, and median funded amounts?

In [235]:
min_funded_amount = df_luxury_loans.funded_amount.min()
max_funded_amount = df_luxury_loans.funded_amount.max()
median_funded_amount = df_luxury_loans.funded_amount.median()
print("min: {}, max: {}, median {}".format(min_funded_amount, max_funded_amount, median_funded_amount))

min: 440000.0, max: 156000000.0, median 1490000.0


2. What are the min, max, and average interest rates?

In [7]:
min_interest_rate = df_luxury_loans['interest rate'].min()
max_interest_rate = df_luxury_loans['interest rate'].max()
average_interest_rate = df_luxury_loans['interest rate'].median()
print("min: {}, max: {}, average {}".format(min_interest_rate, max_interest_rate, average_interest_rate))

min: 0.02648, max: 0.04828, average 0.037875


3. What are the min, max, and median, average property value?

In [8]:
min_propery_value = df_luxury_loans['property value'].min()
max_propery_value = df_luxury_loans['property value'].max()
median_propery_value = df_luxury_loans['property value'].median()
print("min: {}, max: {}, median {}".format(min_propery_value, max_propery_value, median_propery_value))

min: 473200.0, max: 156020900.0, median 1653925.0


4. What are the different “durations months” for the loans and how many loans are under each one?

In [9]:
df_luxury_loans['duration months'].value_counts()

360    1369
180     255
240      48
120       6
Name: duration months, dtype: int64

5. What are the different purposes for the luxury loans and how many loans are there per purpose?

In [10]:
df_luxury_loans.purpose.value_counts()

investment property    536
home                   533
commerical property    520
boat                    62
plane                   27
Name: purpose, dtype: int64

6. What are the 3 most common “total units” found in a property?

In [12]:
df_luxury_loans['TOTAL UNITS'].value_counts()[:3]

1    1189
4     161
6     107
Name: TOTAL UNITS, dtype: int64

7. What are the 3 most common “building class categories” loans are taken for?

In [14]:
df_luxury_loans['BUILDING CLASS CATEGORY'].value_counts()[:3]

15 CONDOS - 2-10 UNIT RESIDENTIAL              466
07 RENTALS - WALKUP APARTMENTS                 367
04 TAX CLASS 1 CONDOS                          348
Name: BUILDING CLASS CATEGORY, dtype: int64

### Level 2: Analyzing (3 points per task)

1. Create bins for property value:
   a. '-1M'
   b. '1M-2M'
   c. '2M-3M'
   d. '3M+'

In [75]:
bins = [0, 1000000, 2000000, 3000000, np.inf]
labels = ['a','b','c','d']
pd.cut(df_luxury_loans['property value'], bins=bins, labels=labels).value_counts()

1M-2M    942
-1M      285
2M-3M    227
3M+      224
Name: property value, dtype: int64

2. Create bins for employment length:
   a. '-5'
   b. '5-10'
   c. '10-15'
   d. '15+'

In [236]:
bins = [0, 5, 10, 15, np.inf]
df_luxury_loans['employment_length'] = df_luxury_loans['employment length']
less_than_five = df_luxury_loans.query('employment_length < 5')
less_than_10 = df_luxury_loans.query('employment_length < 10 and employment_length >= 5').value_counts()
less_than_15 = df_luxury_loans.query('employment_length >= 10 and employment_length < 15').value_counts()
greater_than_15 = df_luxury_loans.query('employment_length >= 15 ').value_counts()

print('-5 {}'.format(len(less_than_five.index)))
print('5-10 {}'.format(len(less_than_10.index)))
print('10-15 {}'.format(len(less_than_15.index)))
print('15+ {}'.format(len(greater_than_15.index)))

# labels = ['-5','5-10','10-15','15+']
# pd.cut(df_luxury_loans['employment length'], bins=bins, labels=labels, include_lowest=True).value_counts()

-5 378
5-10 466
10-15 495
15+ 330


3. Analyze how interest rates differ according to different loan purposes. Visualize this.

In [47]:
interest_rates = df_luxury_loans.groupby('purpose')
interest_rates['interest rate'].median().plot.bar()
interest_rates['interest rate'].median()

purpose
boat                   0.035920
commerical property    0.037825
home                   0.038190
investment property    0.037735
plane                  0.035990
Name: interest rate, dtype: float64

4. Identify the building class category that is most profitable.
    a. Using funded amount and interest rate

In [74]:
df_luxury_loans['profit'] = df_luxury_loans.funded_amount.mul(df_luxury_loans['interest rate'])
df_luxury_loans.groupby('BUILDING CLASS CATEGORY').profit.sum().nlargest(1)


BUILDING CLASS CATEGORY
15 CONDOS - 2-10 UNIT RESIDENTIAL              3.408544e+07
Name: profit, dtype: float64

5. Which "purpose" generates the most value?

In [53]:
df_luxury_loans.groupby('purpose')['property value'].sum().nlargest(1)

purpose
investment property    1.026654e+09
Name: property value, dtype: float64

### Level 3: Modeling

1. Create a model to predict the “property value” (15 points)
   a. Try out at least 3 different models, and comment on why you chose these models 
   b. Measure accuracy (at least one model should have accuracy of 80% or higher) 

Clean up data before modelling

In [173]:
luxury_loan_data = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')
del luxury_loan_data['loan_id']
del luxury_loan_data['funded_date']
del luxury_loan_data['firstname']
del luxury_loan_data['middlename']
del luxury_loan_data['lastname']
del luxury_loan_data['social']
del luxury_loan_data['phone']

bins = [0, 1000000, 2000000, 3000000, np.inf]
labels = ['a','b','c','d']
luxury_loan_data['property value'] = pd.cut(luxury_loan_data['property value'], bins=bins, labels=labels, include_lowest=True)


property = luxury_loan_data['property value']
predict = 'property value'

X = luxury_loan_data.drop([predict], 1)

le = preprocessing.LabelEncoder()
X['purpose'] = le.fit_transform(X.purpose.values)
X['title'] = le.fit_transform(X.title.values)
X['BUILDING CLASS CATEGORY'] = le.fit_transform(X['BUILDING CLASS CATEGORY'].values)
X['TAX CLASS AT PRESENT'] = le.fit_transform(X['TAX CLASS AT PRESENT'].values)
X['BUILDING CLASS AT PRESENT'] = le.fit_transform(X['BUILDING CLASS AT PRESENT'].values)
X['ADDRESS 1'] = le.fit_transform(X['ADDRESS 1'].values)
X['ADDRESS 2'] = le.fit_transform(X['ADDRESS 2'].values)
X['CITY'] = le.fit_transform(X['CITY'].values)
X['STATE'] = le.fit_transform(X['STATE'].values)
X['LAND SQUARE FEET'] = X['LAND SQUARE FEET'].replace(' -  ', 0)
X['GROSS SQUARE FEET'] = X['GROSS SQUARE FEET'].replace(' -  ', 0)

y = property

classes = property.unique()


/var/folders/0m/yz0zvmgd5p9cy9bpc347rm880000gn/T/ipykernel_32335/2622597757.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = luxury_loan_data.drop([predict], 1)


Get our selector and divide the data

In [174]:
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=6)
selector.fit_transform(X, y)

params = selector.get_support()


new_X = X[X.columns[params]]


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(new_X, y, test_size=0.3)

### Get the classifier

Random Forest Classifier

In [175]:
clf = ensemble.RandomForestClassifier(n_estimators=120, max_depth=5)
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)

print(acc * 100)

93.45238095238095


Linear Regression

In [177]:
from sklearn.metrics import mean_squared_error, r2_score

luxury_loan_data = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')
del luxury_loan_data['loan_id']
del luxury_loan_data['funded_date']
del luxury_loan_data['firstname']
del luxury_loan_data['middlename']
del luxury_loan_data['lastname']
del luxury_loan_data['social']
del luxury_loan_data['phone']

property = luxury_loan_data['property value']
predict = 'property value'

X = luxury_loan_data.drop([predict], 1)

le = preprocessing.LabelEncoder()
X['purpose'] = le.fit_transform(X.purpose.values)
X['title'] = le.fit_transform(X.title.values)
X['BUILDING CLASS CATEGORY'] = le.fit_transform(X['BUILDING CLASS CATEGORY'].values)
X['TAX CLASS AT PRESENT'] = le.fit_transform(X['TAX CLASS AT PRESENT'].values)
X['BUILDING CLASS AT PRESENT'] = le.fit_transform(X['BUILDING CLASS AT PRESENT'].values)
X['ADDRESS 1'] = le.fit_transform(X['ADDRESS 1'].values)
X['ADDRESS 2'] = le.fit_transform(X['ADDRESS 2'].values)
X['CITY'] = le.fit_transform(X['CITY'].values)
X['STATE'] = le.fit_transform(X['STATE'].values)
X['LAND SQUARE FEET'] = X['LAND SQUARE FEET'].replace(' -  ', 0)
X['GROSS SQUARE FEET'] = X['GROSS SQUARE FEET'].replace(' -  ', 0)

y = property

classes = property.unique()


from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=6)
selector.fit_transform(X, y)

params = selector.get_support()


new_X = X[X.columns[params]]


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(new_X, y, test_size=0.3)


clf = sklearn.linear_model.LinearRegression()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

# The coefficients
print("Coefficients: \n", clf.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [ 1.11745601 -6.17078755 -0.11270662 -1.45668499  0.69700393 -0.04134281]
Mean squared error: 32917659685.17
Coefficient of determination: 1.00


/var/folders/0m/yz0zvmgd5p9cy9bpc347rm880000gn/T/ipykernel_32335/3196728133.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = luxury_loan_data.drop([predict], 1)


Tweedie Regressor

In [178]:
from sklearn.metrics import mean_squared_error, r2_score

luxury_loan_data = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')
del luxury_loan_data['loan_id']
del luxury_loan_data['funded_date']
del luxury_loan_data['firstname']
del luxury_loan_data['middlename']
del luxury_loan_data['lastname']
del luxury_loan_data['social']
del luxury_loan_data['phone']

property = luxury_loan_data['property value']
predict = 'property value'

X = luxury_loan_data.drop([predict], 1)

le = preprocessing.LabelEncoder()
X['purpose'] = le.fit_transform(X.purpose.values)
X['title'] = le.fit_transform(X.title.values)
X['BUILDING CLASS CATEGORY'] = le.fit_transform(X['BUILDING CLASS CATEGORY'].values)
X['TAX CLASS AT PRESENT'] = le.fit_transform(X['TAX CLASS AT PRESENT'].values)
X['BUILDING CLASS AT PRESENT'] = le.fit_transform(X['BUILDING CLASS AT PRESENT'].values)
X['ADDRESS 1'] = le.fit_transform(X['ADDRESS 1'].values)
X['ADDRESS 2'] = le.fit_transform(X['ADDRESS 2'].values)
X['CITY'] = le.fit_transform(X['CITY'].values)
X['STATE'] = le.fit_transform(X['STATE'].values)
X['LAND SQUARE FEET'] = X['LAND SQUARE FEET'].replace(' -  ', 0)
X['GROSS SQUARE FEET'] = X['GROSS SQUARE FEET'].replace(' -  ', 0)

y = property

classes = property.unique()


from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=6)
selector.fit_transform(X, y)

params = selector.get_support()


new_X = X[X.columns[params]]


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(new_X, y, test_size=0.3)


from sklearn.metrics import mean_squared_error, r2_score

clf = sklearn.linear_model.TweedieRegressor(power=1, alpha=0.5, link='log')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

# The coefficients
print("Coefficients: \n", clf.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [0. 0. 0. 0. 0. 0.]
Mean squared error: 1319337816504.40
Coefficient of determination: -0.01


/var/folders/0m/yz0zvmgd5p9cy9bpc347rm880000gn/T/ipykernel_32335/1258204123.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = luxury_loan_data.drop([predict], 1)
/Users/faaatman/Desktop/GulfBank-Datathon/Datathon/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_per_sample + l2_reg_strength * weights


### Level 3: Modeling

2. create a model to predict the “employment length” (5 points)
   a. Try out at least 3 different models, and comment on why you chose these models 
   b. Measure accuracy (at least one model should have accuracy of 80% or higher)

Clean up data before modelling

In [225]:
luxury_loan_data = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')
del luxury_loan_data['loan_id']
del luxury_loan_data['funded_date']
del luxury_loan_data['firstname']
del luxury_loan_data['middlename']
del luxury_loan_data['lastname']
del luxury_loan_data['social']
del luxury_loan_data['phone']

bins = [0, 5, 10, 15, np.inf]
labels = ['a','b','c','d']
luxury_loan_data['employment length'] = pd.cut(luxury_loan_data['employment length'], bins=bins, labels=labels)


property = luxury_loan_data['employment length']
predict = 'employment length'

X = luxury_loan_data.drop([predict], 1)

le = preprocessing.LabelEncoder()
X['purpose'] = le.fit_transform(X.purpose.values)
X['title'] = le.fit_transform(X.title.values)
X['BUILDING CLASS CATEGORY'] = le.fit_transform(X['BUILDING CLASS CATEGORY'].values)
X['TAX CLASS AT PRESENT'] = le.fit_transform(X['TAX CLASS AT PRESENT'].values)
X['BUILDING CLASS AT PRESENT'] = le.fit_transform(X['BUILDING CLASS AT PRESENT'].values)
X['ADDRESS 1'] = le.fit_transform(X['ADDRESS 1'].values)
X['ADDRESS 2'] = le.fit_transform(X['ADDRESS 2'].values)
X['CITY'] = le.fit_transform(X['CITY'].values)
X['STATE'] = le.fit_transform(X['STATE'].values)
X['LAND SQUARE FEET'] = X['LAND SQUARE FEET'].replace(' -  ', 0)
X['GROSS SQUARE FEET'] = X['GROSS SQUARE FEET'].replace(' -  ', 0)

y = property

classes = property.unique()


/var/folders/0m/yz0zvmgd5p9cy9bpc347rm880000gn/T/ipykernel_32335/3992658464.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = luxury_loan_data.drop([predict], 1)


Get our selector and divide the data

In [228]:
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=6)
selector.fit_transform(X, y)

params = selector.get_support()


new_X = X[X.columns[params]]


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(new_X, y, test_size=0.1, stratify=y, shuffle=True )

### Get the classifier

Random Forest Classifier

In [234]:
# clf = ensemble.RandomForestClassifier(n_estimators=25, max_depth=3)
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)

print(acc * 100)

28.57142857142857


Linear Regression

In [197]:
from sklearn.metrics import mean_squared_error, r2_score

luxury_loan_data = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')
del luxury_loan_data['loan_id']
del luxury_loan_data['funded_date']
del luxury_loan_data['firstname']
del luxury_loan_data['middlename']
del luxury_loan_data['lastname']
del luxury_loan_data['social']
del luxury_loan_data['phone']

property = luxury_loan_data['employment length']
predict = 'employment length'

X = luxury_loan_data.drop([predict], 1)

le = preprocessing.LabelEncoder()
X['purpose'] = le.fit_transform(X.purpose.values)
X['title'] = le.fit_transform(X.title.values)
X['BUILDING CLASS CATEGORY'] = le.fit_transform(X['BUILDING CLASS CATEGORY'].values)
X['TAX CLASS AT PRESENT'] = le.fit_transform(X['TAX CLASS AT PRESENT'].values)
X['BUILDING CLASS AT PRESENT'] = le.fit_transform(X['BUILDING CLASS AT PRESENT'].values)
X['ADDRESS 1'] = le.fit_transform(X['ADDRESS 1'].values)
X['ADDRESS 2'] = le.fit_transform(X['ADDRESS 2'].values)
X['CITY'] = le.fit_transform(X['CITY'].values)
X['STATE'] = le.fit_transform(X['STATE'].values)
X['LAND SQUARE FEET'] = X['LAND SQUARE FEET'].replace(' -  ', 0)
X['GROSS SQUARE FEET'] = X['GROSS SQUARE FEET'].replace(' -  ', 0)

y = property

classes = property.unique()


from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=6)
selector.fit_transform(X, y)

params = selector.get_support()


new_X = X[X.columns[params]]


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(new_X, y, test_size=0.3)


clf = sklearn.linear_model.LinearRegression()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

# The coefficients
print("Coefficients: \n", clf.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-9.64653212e-07 -6.68358166e-06  4.73016987e-07  6.03113737e-07
  1.46229919e-04 -8.03539543e-05]
Mean squared error: 26.62
Coefficient of determination: 0.00


/var/folders/0m/yz0zvmgd5p9cy9bpc347rm880000gn/T/ipykernel_32335/881175782.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = luxury_loan_data.drop([predict], 1)


Tweedie Regressor

In [198]:
from sklearn.metrics import mean_squared_error, r2_score

luxury_loan_data = pd.read_csv('data-set/LuxuryLoanPortfolio.csv')
del luxury_loan_data['loan_id']
del luxury_loan_data['funded_date']
del luxury_loan_data['firstname']
del luxury_loan_data['middlename']
del luxury_loan_data['lastname']
del luxury_loan_data['social']
del luxury_loan_data['phone']

property = luxury_loan_data['employment length']
predict = 'employment length'

X = luxury_loan_data.drop([predict], 1)

le = preprocessing.LabelEncoder()
X['purpose'] = le.fit_transform(X.purpose.values)
X['title'] = le.fit_transform(X.title.values)
X['BUILDING CLASS CATEGORY'] = le.fit_transform(X['BUILDING CLASS CATEGORY'].values)
X['TAX CLASS AT PRESENT'] = le.fit_transform(X['TAX CLASS AT PRESENT'].values)
X['BUILDING CLASS AT PRESENT'] = le.fit_transform(X['BUILDING CLASS AT PRESENT'].values)
X['ADDRESS 1'] = le.fit_transform(X['ADDRESS 1'].values)
X['ADDRESS 2'] = le.fit_transform(X['ADDRESS 2'].values)
X['CITY'] = le.fit_transform(X['CITY'].values)
X['STATE'] = le.fit_transform(X['STATE'].values)
X['LAND SQUARE FEET'] = X['LAND SQUARE FEET'].replace(' -  ', 0)
X['GROSS SQUARE FEET'] = X['GROSS SQUARE FEET'].replace(' -  ', 0)

y = property

classes = property.unique()


from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=6)
selector.fit_transform(X, y)

params = selector.get_support()


new_X = X[X.columns[params]]


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(new_X, y, test_size=0.3)


from sklearn.metrics import mean_squared_error, r2_score

clf = sklearn.linear_model.TweedieRegressor(power=1, alpha=0.5, link='log')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

# The coefficients
print("Coefficients: \n", clf.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

/var/folders/0m/yz0zvmgd5p9cy9bpc347rm880000gn/T/ipykernel_32335/4018532270.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = luxury_loan_data.drop([predict], 1)


Coefficients: 
 [0. 0. 0. 0. 0. 0.]
Mean squared error: 26.20
Coefficient of determination: -0.00


/Users/faaatman/Desktop/GulfBank-Datathon/Datathon/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_per_sample + l2_reg_strength * weights
